In [127]:
import pandas as pd
import numpy as np
import random

import seaborn as sns
import matplotlib.pyplot as plt

from solver import Instance, Solver
from argparse import Namespace
from gurobipy import Model, GRB, tupledict
import json

import ast

In [128]:
read_instance_file = True
city = "berlin"

In [129]:
if read_instance_file:
    demand_baseline = "1.00"
    demand_type = "uniform"
    
    demand_file = f'{city}_db={demand_baseline}_dt={demand_type}.csv'
    print(demand_file)

    demand_df = pd.read_csv(demand_file, index_col=0)
    demand_df['area_id'] = demand_df['area_id'].astype(str) 
    days_ = []
    for i in range(7):
        days_.append([i, "1.00", "uniform"])
    days = pd.DataFrame(days_, columns=['day', 'demand_baseline', 'demand_type']) 
else:
    # Generate instance file
    # Generate several days instances 
    days = pd.DataFrame([
        [0, "1.00", "uniform"], # Monday
        [1, "1.00", "uniform"], # Tuesday
        [2, "1.00", "uniform"], # Wednesday
        [3, "1.00", "uniform"], # Thursday
        [4, "1.00", "uniform"], # Friday
        [5, "1.00", "uniform"], # Satuday
        [6, "1.00", "uniform"], # Sunday
    ], columns=['day', 'demand_baseline', 'demand_type']
    )

    demand = []
    area_region_map = None

    for i, row in days.iterrows():
        demand_baseline, demand_type = row['demand_baseline'], row['demand_type']
        key = f'{city}_db={demand_baseline}_dt={demand_type}'

        instance_file = f'../instances/{city}_db={demand_baseline}_dt={demand_type}.json'
        with open(instance_file, 'r') as file:
            instance_data = json.load(file)

        if area_region_map is None:
            regions = instance_data['geography']['city']['regions']
            area_region_map = {}
            for region in regions:
                areas = region['areas'] 
                for area in areas:
                    area_region_map[area['id']] = region['id']
        
        scenario = random.randint(0, instance_data['num_scenarios']-1)
        data_ = (
            pd.DataFrame(instance_data['scenarios'][scenario]['data'])
            .assign(
                day = row['day']
            )
        )
        demand.append(data_)

    demand_df = pd.concat(demand)
    # Add region
    demand_df['region_id'] = demand_df['area_id'].map(area_region_map)

    demand_df = demand_df[['day', 'region_id', 'area_id', 'demand', 'required_couriers']]
    #demand_df.to_csv(f'{city}_db={demand_baseline}_dt={demand_type}.csv')

display(demand_df.head(20))

berlin_db=1.00_dt=uniform.csv


,day,region_id,area_id,demand,required_couriers
0,0,1,10115,"[0, 2, 2, 4, 1, 4, 7, 2]","[0, 1, 1, 1, 1, 1, 2, 1]"
1,0,1,10117,"[4, 3, 1, 1, 2, 2, 2, 3]","[1, 1, 1, 1, 1, 1, 1, 1]"
2,0,1,10119,"[0, 2, 0, 0, 2, 2, 6, 4]","[0, 1, 0, 0, 1, 1, 2, 1]"
3,0,1,10178,"[3, 1, 2, 0, 4, 2, 0, 0]","[1, 1, 1, 0, 1, 1, 0, 0]"
4,0,1,10179,"[3, 1, 3, 2, 1, 5, 1, 1]","[1, 1, 1, 1, 1, 1, 1, 1]"
5,0,2,10243,"[4, 4, 2, 2, 3, 2, 1, 3]","[1, 1, 1, 1, 1, 1, 1, 1]"
6,0,0,10245,"[3, 2, 3, 5, 2, 1, 1, 2]","[1, 1, 1, 1, 1, 1, 1, 1]"
7,0,0,10247,"[4, 4, 8, 7, 3, 4, 7, 4]","[1, 1, 2, 2, 1, 1, 2, 1]"
8,0,0,10249,"[4, 4, 1, 5, 2, 1, 3, 5]","[1, 1, 1, 1, 1, 1, 1, 1]"
9,0,0,10315,"[4, 8, 4, 1, 3, 5, 2, 5]","[1, 2, 1, 1, 1, 1, 1, 1]"


In [164]:
demand_df.query('area_id == "10115"')

,day,region_id,area_id,demand,required_couriers
0,0,1,10115,"[0, 2, 2, 4, 1, 4, 7, 2]","[0, 1, 1, 1, 1, 1, 2, 1]"
0,1,1,10115,"[5, 1, 3, 2, 4, 0, 3, 2]","[1, 1, 1, 1, 1, 0, 1, 1]"
0,2,1,10115,"[1, 4, 5, 2, 3, 1, 4, 3]","[1, 1, 1, 1, 1, 1, 1, 1]"
0,3,1,10115,"[0, 2, 2, 4, 1, 4, 7, 2]","[0, 1, 1, 1, 1, 1, 2, 1]"
0,4,1,10115,"[4, 2, 6, 1, 0, 2, 1, 0]","[1, 1, 2, 1, 0, 1, 1, 0]"
0,5,1,10115,"[4, 3, 1, 3, 2, 3, 0, 2]","[1, 1, 1, 1, 1, 1, 0, 1]"
0,6,1,10115,"[2, 3, 0, 2, 3, 3, 2, 0]","[1, 1, 0, 1, 1, 1, 1, 0]"


In [130]:
scheduling_solution = demand_df.copy().reset_index(drop=True)
scheduling_solution['couriers_per_region'] = ''
scheduling_solution['hired_couriers_scheduling'] = ''

In [131]:
# READ OPTIMAL SCHEDULING SOLUTIONS
outsourcing_cost_multiplier = 1.5
arguments = {
    'model': 'base', # choices=('base', 'fixed', 'partflex', 'flex')
    #'instance': instance_file,
    'outsourcing_cost_multiplier': outsourcing_cost_multiplier,
    'regional_multiplier': 2.0,
    'global_multiplier': 3.0,
    'max_n_shifts': 10,
    'output': 'output.txt'
}

for i, row in days[['demand_baseline', 'demand_type']].drop_duplicates().iterrows():
    demand_baseline, demand_type = row['demand_baseline'], row['demand_type'] 
    instance_file = f'../instances/{city}_db={demand_baseline}_dt={demand_type}.json'

    arguments['instance'] = instance_file
    args = Namespace(**arguments)

    i = Instance(args=args)
    solver = Solver(args=args, i=i)

    scheduling_best_value, x, i = solver.return_solve_base()

    # Number of couriers in a region
    n_couriers_per_region = '[' + ', '.join([str(i.ub_reg[region]) for region in i.regions]) + ']'
    selected_days = list(days.query(f'demand_baseline == "{demand_baseline}" & demand_type == "{demand_type}"')['day'].values)
    scheduling_solution.loc[ scheduling_solution['day'].isin(selected_days), 'couriers_per_region'] = n_couriers_per_region

    hired_couriers = {
        a: [int(x[a, theta].X) for theta in i.periods] for a in i.areas
    }
    scheduling_solution.loc[ scheduling_solution['day'].isin(selected_days), 'hired_couriers_scheduling'] = \
        scheduling_solution.loc[ scheduling_solution['day'].isin(selected_days), 'area_id'].map(hired_couriers)

Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (mac64[rosetta2])

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 14200 rows, 14632 columns and 24472 nonzeros
Model fingerprint: 0x5b303499
Variable types: 14160 continuous, 472 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [3e-02, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+00, 3e+02]
Found heuristic solution: objective 1421.0500000
Presolve removed 14178 rows and 14565 columns
Presolve time: 0.09s
Presolved: 22 rows, 67 columns, 88 nonzeros
Found heuristic solution: objective 560.3250000
Variable types: 46 continuous, 21 integer (0 binary)

Root relaxation: objective 5.363750e+02, 21 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0        

In [162]:
scheduling_best_value*7

3754.625

In [132]:
scheduling_solution.head(20)

,day,region_id,area_id,demand,required_couriers,couriers_per_region,hired_couriers_scheduling
0,0,1,10115,"[0, 2, 2, 4, 1, 4, 7, 2]","[0, 1, 1, 1, 1, 1, 2, 1]","[33, 22, 20, 25]","[1, 1, 1, 1, 1, 1, 1, 1]"
1,0,1,10117,"[4, 3, 1, 1, 2, 2, 2, 3]","[1, 1, 1, 1, 1, 1, 1, 1]","[33, 22, 20, 25]","[1, 1, 1, 1, 1, 1, 1, 1]"
2,0,1,10119,"[0, 2, 0, 0, 2, 2, 6, 4]","[0, 1, 0, 0, 1, 1, 2, 1]","[33, 22, 20, 25]","[1, 1, 1, 1, 1, 1, 1, 1]"
3,0,1,10178,"[3, 1, 2, 0, 4, 2, 0, 0]","[1, 1, 1, 0, 1, 1, 0, 0]","[33, 22, 20, 25]","[1, 1, 1, 1, 1, 1, 1, 1]"
4,0,1,10179,"[3, 1, 3, 2, 1, 5, 1, 1]","[1, 1, 1, 1, 1, 1, 1, 1]","[33, 22, 20, 25]","[1, 1, 1, 1, 1, 1, 1, 1]"
5,0,2,10243,"[4, 4, 2, 2, 3, 2, 1, 3]","[1, 1, 1, 1, 1, 1, 1, 1]","[33, 22, 20, 25]","[1, 1, 1, 1, 1, 1, 1, 1]"
6,0,0,10245,"[3, 2, 3, 5, 2, 1, 1, 2]","[1, 1, 1, 1, 1, 1, 1, 1]","[33, 22, 20, 25]","[1, 1, 1, 1, 1, 1, 1, 1]"
7,0,0,10247,"[4, 4, 8, 7, 3, 4, 7, 4]","[1, 1, 2, 2, 1, 1, 2, 1]","[33, 22, 20, 25]","[1, 1, 1, 1, 2, 2, 2, 1]"
8,0,0,10249,"[4, 4, 1, 5, 2, 1, 3, 5]","[1, 1, 1, 1, 1, 1, 1, 1]","[33, 22, 20, 25]","[1, 1, 1, 1, 2, 1, 1, 1]"
9,0,0,10315,"[4, 8, 4, 1, 3, 5, 2, 5]","[1, 2, 1, 1, 1, 1, 1, 1]","[33, 22, 20, 25]","[1, 2, 1, 1, 1, 1, 1, 1]"


In [133]:
# Calculate costs (benchmark)
if False: # copy that
    m.setObjective(
        sum([
            sum([cost_couriers[a, theta, d] * k_var[e, a, theta, d] for e in E[r]]) 
                + omega_var[a, theta, d]
        for r in R for a in A[r] for theta in Theta for d in D]
        )
    )
    factor = deliveries[a, theta, d] / couriers_needed[a, theta, d] * c_out 

    ((couriers_needed[a, theta, d] - sum([k_var[e, a, theta, d] for e in E[r]])) * factor <= omega_var[a, theta, d])

cost_courier = 1
scheduling_solution['hiring_costs'] = 0
scheduling_solution['outsourcing_costs'] = 0

for i_, row in scheduling_solution.iterrows():
    demand = ast.literal_eval(row['demand'])
    required_couriers = ast.literal_eval(row['required_couriers'])
    hired_couriers = row['hired_couriers_scheduling']

    hiring, outsourcing = 0, 0
    for demand_, required_, hired_ in zip(demand, required_couriers, hired_couriers):
        if hired_ < required_:
            factor = demand_ / required_ * outsourcing_cost_multiplier
            outsourcing += (required_ - hired_) * factor
        hiring += hired_*cost_courier
    
    scheduling_solution.loc[i_, 'hiring_costs'] = hiring
    scheduling_solution.loc[i_, 'outsourcing_costs'] = outsourcing

scheduling_solution.head(10)

,day,region_id,area_id,demand,required_couriers,couriers_per_region,hired_couriers_scheduling,hiring_costs,outsourcing_costs
0,0,1,10115,"[0, 2, 2, 4, 1, 4, 7, 2]","[0, 1, 1, 1, 1, 1, 2, 1]","[33, 22, 20, 25]","[1, 1, 1, 1, 1, 1, 1, 1]",8,5.25
1,0,1,10117,"[4, 3, 1, 1, 2, 2, 2, 3]","[1, 1, 1, 1, 1, 1, 1, 1]","[33, 22, 20, 25]","[1, 1, 1, 1, 1, 1, 1, 1]",8,0.00
2,0,1,10119,"[0, 2, 0, 0, 2, 2, 6, 4]","[0, 1, 0, 0, 1, 1, 2, 1]","[33, 22, 20, 25]","[1, 1, 1, 1, 1, 1, 1, 1]",8,4.50
3,0,1,10178,"[3, 1, 2, 0, 4, 2, 0, 0]","[1, 1, 1, 0, 1, 1, 0, 0]","[33, 22, 20, 25]","[1, 1, 1, 1, 1, 1, 1, 1]",8,0.00
4,0,1,10179,"[3, 1, 3, 2, 1, 5, 1, 1]","[1, 1, 1, 1, 1, 1, 1, 1]","[33, 22, 20, 25]","[1, 1, 1, 1, 1, 1, 1, 1]",8,0.00
5,0,2,10243,"[4, 4, 2, 2, 3, 2, 1, 3]","[1, 1, 1, 1, 1, 1, 1, 1]","[33, 22, 20, 25]","[1, 1, 1, 1, 1, 1, 1, 1]",8,0.00
6,0,0,10245,"[3, 2, 3, 5, 2, 1, 1, 2]","[1, 1, 1, 1, 1, 1, 1, 1]","[33, 22, 20, 25]","[1, 1, 1, 1, 1, 1, 1, 1]",8,0.00
7,0,0,10247,"[4, 4, 8, 7, 3, 4, 7, 4]","[1, 1, 2, 2, 1, 1, 2, 1]","[33, 22, 20, 25]","[1, 1, 1, 1, 2, 2, 2, 1]",11,11.25
8,0,0,10249,"[4, 4, 1, 5, 2, 1, 3, 5]","[1, 1, 1, 1, 1, 1, 1, 1]","[33, 22, 20, 25]","[1, 1, 1, 1, 2, 1, 1, 1]",9,0.00
9,0,0,10315,"[4, 8, 4, 1, 3, 5, 2, 5]","[1, 2, 1, 1, 1, 1, 1, 1]","[33, 22, 20, 25]","[1, 2, 1, 1, 1, 1, 1, 1]",9,0.00


In [134]:
# TOTAL COSTS
print(f'Hiring costs: {scheduling_solution["hiring_costs"].sum()}') # They are not right!!!
print(f'Outsourcing costs: {scheduling_solution["outsourcing_costs"].sum()}')
print(f'Total costs: {scheduling_solution["hiring_costs"].sum() + scheduling_solution["outsourcing_costs"].sum()}')

Hiring costs: 3206
Outsourcing costs: 592.5
Total costs: 3798.5


In [166]:
scheduling_solution['couriers_per_region'].unique()[0]

'[33, 22, 20, 25]'

In [135]:
# PARAMETERS
# Regions
R = sorted(list(demand_df['region_id'].unique()))

region_area_map = {}
for r in R:
    for a in list(demand_df.query(f'region_id == {r}')['area_id'].unique()):
        region_area_map[a] = r
        
# Areas 
A = {}
area_map = {a: i for i, a in enumerate(list(demand_df['area_id'].unique())) }
areas = list(area_map.keys())
n_areas = len(areas)
for r in R:
    A[r] = [ area_map[a] for a in list(demand_df.query(f'region_id == {r}')['area_id'].unique()) ]

# Employees
if False:
    n_employees = 12
    employees = [e for e in range(n_employees)]
    E = {}
    #for r in R:
    #    E[r] = employees
    E[0] = [0,1,2]
    E[1] = [3,4,5]
    E[2] = [6,7,8]
    E[3] = [9,10,11]
else:
    # Read employees from scheduling
    E = {}
    n_empl_init = 0
    for r, n_empl in enumerate(ast.literal_eval(scheduling_solution['couriers_per_region'].unique()[0])):
        E[r] = [i for i in range(n_empl_init, n_empl_init+n_empl)]
        n_empl_init = n_empl
    
    def flatten(xss):
        return [x for xs in xss for x in xs]

    n_employees = len(flatten(E.values()))
    employees = [e for e in range(n_employees)]
 
# Set of all shifts available
P = {}
n_shifts = 2
shifts = [s for s in range(n_shifts)]
for r in R:
    P[r] = shifts

# Shifts start and end times
shifts_start = {0: 0, 1:2}
shifts_end = {0: 4, 1:6}

# Periods
n_periods = 8
Theta = [i for i in range(n_periods)]

# Days
n_days = 7
D = [d for d in range(n_days)]

# Set of demand scenarios
# TODO

# Hours in shift
h = {}
n_hours = 8
for p in range(n_shifts):
    h[p] = n_hours

# Cost outsource
c_out = outsourcing_cost_multiplier

# Number of deliveries to perform (n_{a theta d})
deliveries = np.zeros((n_areas, n_periods, n_days))
for i, a in enumerate(areas):
    for k, d in enumerate(D):
        period_demands = demand_df.query(f'area_id == "{a}" & day == {d}')
        deliveries[i, :, k] = ast.literal_eval(period_demands['demand'].values[0])

# Number of couriers needed (m_{a theta d})
couriers_needed = np.zeros((n_areas, n_periods, n_days))
for i, a in enumerate(areas):
    for k, d in enumerate(D):
        period_demands = demand_df.query(f'area_id == "{a}" & day == {d}')
        couriers_needed[i, :, k] = ast.literal_eval(period_demands['required_couriers'].values[0])

# Cost of employed courier (c_{a theta d})
cost_couriers = np.zeros((n_areas, n_periods, n_days))
cost_courier = 1
for i, a in enumerate(areas):
    for j, t in enumerate(Theta):
        for k, d in enumerate(D):
            cost_couriers[i, j, k] = cost_courier

# Min hours worked for employee e
min_hours_worked = 8
h_min = {e: min_hours_worked for e in employees}

# Max hours worked for employee e
max_hours_worked = 8*6
h_max = {e: max_hours_worked for e in employees}

# Max differing starts
max_unique_starts = 2
b_max = {e: max_unique_starts for e in employees}

# mega_value
M = 999_999

# DECISION VARIABLES

In [136]:
m = Model()

# r_{e p d}
r_var = m.addVars(n_employees, n_shifts, n_days, vtype=GRB.BINARY, name='r')

# k_{e a theta d}
k_var = m.addVars(n_employees, n_areas, n_periods, n_days, vtype=GRB.BINARY, name='k')

# U_{e p}
u_var = m.addVars(n_employees, n_shifts, vtype=GRB.BINARY, name='u')

# omega_{a theta d}
omega_var = m.addVars(n_areas, n_periods, n_days, vtype=GRB.CONTINUOUS, lb=0, name='omega')

# CONSTRAINTS

In [137]:
# 1. Connecting employees moving around areas to shift assignment p
if False:
    for r in R:
        for e in E[r]:
            for p in P[r]:
                for d in D:
                    Ar = A[r]
                    m.addConstr(
                        (sum([k_var[e, a, theta, d] for a in Ar for theta in Theta]) == 1/2 * h[p] * r_var[e,p,d] ),
                        name = f'moving_areas_{r}_{e}_{p}_{d}'          
                    )

# CORRECTED!
for r in R:
    for e in E[r]:
        for d in D:
            Ar, Pr = A[r], P[r]
            m.addConstr(
                (sum([k_var[e, a, theta, d] for a in Ar for theta in Theta]) == 1/2 * sum([h[p] * r_var[e,p,d] for p in Pr])),
                name = f'moving_areas_{r}_{e}_{d}'          
            )

In [138]:
# 2. Employee can only be assigned to one area at a time 
for r in R:
    for e in E[r]:
        for theta in Theta:
            for d in D:
                Ar = A[r]
                m.addConstr((sum([k_var[e, a, theta, d] for a in Ar]) <= 1),
                    name = f'employee_{r}_{e}_{theta}_{d}'          
                )

# ADDED: They only can be assigned one region!
#for e in employees:
#    for theta in Theta:
#        for d in D:
#            m.addConstr((sum([k_var[e, area_map[a], theta, d] for a in areas]) <= 1),
#                name = f'only_one_area_{e}_{theta}_{d}'          
#            )

In [139]:
# ADDED - Shift start and end times
for shift in shifts:
    start = shifts_start[shift]
    for theta in Theta[:start]:
        for e in employees:
            for a in areas:
                for d in D:
                    m.addConstr((k_var[e, area_map[a], theta, d] * r_var[e, shift, d] == 0),
                        name = f'start_time_{e}_{a}_{shift}_{d}'          
                    )

for shift in shifts:
    end = shifts_end[shift]
    for theta in Theta[end:]:
        for e in employees:
            for a in areas:
                for d in D:
                    m.addConstr((k_var[e, area_map[a], theta, d] * r_var[e, shift, d] == 0),
                        name = f'end_time_{e}_{a}_{shift}_{d}'          
                    )

In [140]:
# 3. Employee can only work one shift a day
for r in R:
    for es in E[r]:
        for d in D:
            Pr = P[r]
            m.addConstr((sum([r_var[e, p, d] for p in Pr]) <= 1),
                name = f'only_one_shift_{r}_{e}_{theta}_{d}'          
            )

In [141]:
# 4. One rest day a week
for r in R:
    for e in E[r]:
        Pr = P[r]
        m.addConstr((sum([r_var[e, p, d] for p in Pr for d in D]) <= 6),
            name = f'rest_day_{r}_{e}'          
        )

In [142]:
# 5. Min - Max hours worked per week
for r in R:
    Pr = P[r]
    for e in E[r]:
        min_hours = h_min[e]
        max_hours = h_max[e]

        m.addConstr((sum([ h[p] * r_var[e, p, d] for p in Pr for d in D]) >= min_hours),
            name = f'min_hours_{r}_{e}'          
        )

        m.addConstr(( sum([ h[p] * r_var[e, p, d] for p in Pr for d in D]) <= max_hours),
            name = f'max_hours_{r}_{e}'          
        )

In [143]:
# 6. Different shifting times constraint
for r in R:
    for e in E[r]:
        for p in P[r]:
            m.addConstr((sum([r_var[e, p, d] for d in D]) <= u_var[e, p] * M),
                name = f'start_times_{r}_{e}_1'          
            )

for r in R:
    for e in E[r]:
        Pr = P[r]
        m.addConstr((sum([u_var[e, p] for p in Pr]) <= b_max[e]),
            name = f'start_times_{r}_{e}_2'          
        )

In [144]:
# 7. Employee can't work then outsource
for r in R:
    for a in A[r]:
        for theta in Theta:
            for d in D:
                # Be careful with parantheses!!
                if couriers_needed[a,theta,d] > 0:
                    factor = deliveries[a, theta, d] / couriers_needed[a, theta, d] * c_out 
                else:
                    factor = 0
                m.addConstr(
                    ((couriers_needed[a, theta, d] - sum([k_var[e, a, theta, d] for e in E[r]])) * factor <= omega_var[a, theta, d]),
                    name = f'outsource_{r}_{a}_{theta}_{d}'   
                )

# OBJECTIVE FUNCTION

In [145]:
factor_s = 1
m.setObjective(
    sum([
        sum([cost_couriers[a, theta, d] * k_var[e, a, theta, d] for e in E[r]]) 
            + omega_var[a, theta, d]
    for r in R for a in A[r] for theta in Theta for d in D]
    )
)

In [146]:
m.ModelSense = GRB.MINIMIZE
m.optimize()

Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (mac64[rosetta2])

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 10904 rows, 335304 columns and 240536 nonzeros
Model fingerprint: 0x45a75171
Model has 330400 quadratic constraints
Variable types: 3304 continuous, 332000 integer (332000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+06]
  QMatrix range    [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+01]
Presolve removed 3867 rows and 282556 columns
Presolve time: 2.03s
Presolved: 7037 rows, 52748 columns, 157620 nonzeros
Variable types: 0 continuous, 52748 integer (52663 binary)
Deterministic concurrent LP optimizer: primal and dual simplex
Showing first log only...


Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.5260000e+03   3.000000e+00   4.199440e+07      6s
       6    1.047400

# SOLUTION

In [147]:
# General values


In [148]:
# Total couriers in day
couriers_needed_ = []
for d in D:
    data = (
        pd.DataFrame(couriers_needed[:,:,d], index=areas, columns=Theta).reset_index()
        .rename(columns={'index': 'area'})
        #.assign(day = d)
    )

    area_day_data = (
        pd.melt(data, id_vars=['area'], value_vars=set(data.columns).difference('area'))
        .rename(columns={'variable': 'period', 'value': 'n_couriers'})
        .assign(day = d)
    )
    couriers_needed_.append(area_day_data)

couriers_needed_df = pd.concat(couriers_needed_)
couriers_needed_df.head()

,area,period,n_couriers,day
0,10115,0,0.0,0
1,10117,0,1.0,0
2,10119,0,0.0,0
3,10178,0,1.0,0
4,10179,0,1.0,0


In [149]:
# Demand
deliveries_ = []
for d in D:
    data = (
        pd.DataFrame(deliveries[:,:,d], index=areas, columns=Theta).reset_index()
        .rename(columns={'index': 'area'})
        #.assign(day = d)
    )

    area_day_data = (
        pd.melt(data, id_vars=['area'], value_vars=set(data.columns).difference('area'))
        .rename(columns={'variable': 'period', 'value': 'deliveries'})
        .assign(day = d)
    )
    deliveries_.append(area_day_data)

deliveries_df = pd.concat(deliveries_)
deliveries_df.head(10)

,area,period,deliveries,day
0,10115,0,0.0,0
1,10117,0,4.0,0
2,10119,0,0.0,0
3,10178,0,3.0,0
4,10179,0,3.0,0
5,10243,0,4.0,0
6,10245,0,3.0,0
7,10247,0,4.0,0
8,10249,0,4.0,0
9,10315,0,4.0,0


In [150]:
couriers_needed_df = pd.merge(couriers_needed_df, deliveries_df, on=['area', 'period', 'day'])

In [151]:
# Employee shifts
employees_shifts = []
for e in employees:
    for p in range(n_shifts):
        for d in D:
            value = r_var[e, p, d].X
            if value > 0.5:
                hours_worked = h[p]
                employees_shifts.append([e, p, d, hours_worked])

employees_shifts_df = pd.DataFrame(employees_shifts, columns=['employee', 'shift', 'day', 'hours_worked'])
employees_shifts_df.head(10)

,employee,shift,day,hours_worked
0,0,0,1,8
1,0,0,4,8
2,0,0,5,8
3,0,1,0,8
4,0,1,2,8
5,1,0,0,8
6,2,0,2,8
7,2,0,4,8
8,2,0,6,8
9,2,1,1,8


In [152]:
# Total hours worked
#employees_shifts_df.groupby('employee')['hours_worked'].sum()

In [153]:
# Area Employee assignment
area_employee_assignment = []
for e in employees:
    for a in areas:
        for theta in Theta:
            for d in D:
                for p in shifts:
                    shift = int(r_var[e, p, d].X)
                value = k_var[e, area_map[a], theta, d].X
                if value > 0.5:
                    area_employee_assignment.append([e, a, theta, d, shift])

area_employee_assignment_df = pd.DataFrame(area_employee_assignment, columns=['employee', 'area', 'period', 'day', 'shift'])
area_employee_assignment_df.head(10)

,employee,area,period,day,shift
0,0,10245,2,1,0
1,0,10245,2,5,0
2,0,10245,3,0,1
3,0,10247,1,1,0
4,0,10247,2,0,1
5,0,10247,3,4,0
6,0,10249,0,4,0
7,0,10249,1,5,0
8,0,10249,3,5,0
9,0,10249,4,0,1


In [154]:
employees_shifts_df.query('employee == 0').sort_values(['day'])

,employee,shift,day,hours_worked
3,0,1,0,8
0,0,0,1,8
4,0,1,2,8
1,0,0,4,8
2,0,0,5,8


In [155]:
(
    area_employee_assignment_df
    .query('employee == 0')
    .merge(couriers_needed_df, on=['day', 'area', 'period'])
    .assign(
        region = area_employee_assignment_df['area'].map(region_area_map)
    )
    .sort_values(['day', 'period'])
    [['employee', 'shift', 'day', 'period', 'area', 'region', 'n_couriers', 'deliveries']]
    .head(100)
)

,employee,shift,day,period,area,region,n_couriers,deliveries
4,0,1,0,2,10247,0,2.0,8.0
2,0,1,0,3,10245,0,1.0,5.0
9,0,1,0,4,10249,0,1.0,2.0
14,0,1,0,5,10319,0,1.0,2.0
10,0,0,1,0,10315,0,1.0,3.0
3,0,0,1,1,10247,0,1.0,3.0
0,0,0,1,2,10245,0,2.0,6.0
13,0,0,1,3,10318,0,1.0,3.0
17,0,1,2,2,10367,0,1.0,3.0
19,0,1,2,3,10369,0,1.0,2.0


In [156]:
(
    area_employee_assignment_df
    .query('employee == 1')
    #.merge(couriers_needed_df, on=['day', 'area', 'period'])
    .assign(
        region = area_employee_assignment_df['area'].map(region_area_map)
    )
    .sort_values(['day', 'period'])
    [['employee', 'shift', 'day', 'period', 'area', 'region']] #, 'n_couriers']]
    .head(100)
)

,employee,shift,day,period,area,region
22,1,0,0,0,10317,0
21,1,0,0,1,10315,0
20,1,0,0,2,10247,0
23,1,0,0,3,10319,0


In [157]:
# areas
area_period_days_df = (
    area_employee_assignment_df
    .groupby(['area', 'period', 'day'])
    .agg({'employee': ['count', 'unique']})
    .reset_index()
)
area_period_days_df.columns = ['area', 'period', 'day', 'employee_count', 'employees_assign']
area_period_days_df

,area,period,day,employee_count,employees_assign
0,10115,0,1,1,[37]
1,10115,0,4,1,[46]
2,10115,0,5,1,[51]
3,10115,0,6,1,[49]
4,10115,1,0,1,[47]
...,...,...,...,...,...
1678,10999,5,2,1,[28]
1679,10999,5,3,1,[24]
1680,10999,5,4,1,[34]
1681,10999,5,5,1,[33]


In [158]:
# Outsourcing
outsourcing_shifts = []
for a in areas:
    for theta in Theta:
        for d in D:
            value = omega_var[area_map[a],theta,d].X
            #print(f'{a} {theta} {d} : {value}')
            if value > 0.0:
                outsourcing_shifts.append([a, theta, d, value])

outsourcing_shifts_df = pd.DataFrame(outsourcing_shifts, columns=['area', 'period', 'day', 'cost_outsource'])
outsourcing_shifts_df

,area,period,day,cost_outsource
0,10115,0,2,1.5
1,10115,4,0,1.5
2,10115,4,3,1.5
3,10115,5,2,1.5
4,10115,6,0,10.5
...,...,...,...,...
1019,10999,7,2,7.5
1020,10999,7,3,3.0
1021,10999,7,4,3.0
1022,10999,7,5,6.0


In [159]:
# Join all
whole_solution_df = (
    couriers_needed_df
    # Employees
    .merge(area_period_days_df,  on=['area', 'period', 'day'], how='left')
    # Outsource
    .merge(outsourcing_shifts_df, on=['area', 'period', 'day'], how='left')
)
whole_solution_df.head(30)

,area,period,n_couriers,day,deliveries,employee_count,employees_assign,cost_outsource
0,10115,0,0.0,0,0.0,NaN,NaN,NaN
1,10117,0,1.0,0,4.0,1.0,[41],NaN
2,10119,0,0.0,0,0.0,NaN,NaN,NaN
3,10178,0,1.0,0,3.0,1.0,[38],NaN
4,10179,0,1.0,0,3.0,1.0,[52],NaN
5,10243,0,1.0,0,4.0,1.0,[30],NaN
6,10245,0,1.0,0,3.0,1.0,[30],NaN
7,10247,0,1.0,0,4.0,1.0,[25],NaN
8,10249,0,1.0,0,4.0,1.0,[31],NaN
9,10315,0,1.0,0,4.0,1.0,[27],NaN


In [160]:
whole_solution_df.query('n_couriers == 2')

,area,period,n_couriers,day,deliveries,employee_count,employees_assign,cost_outsource
10,10317,0,2.0,0,6.0,2.0,"[1, 26]",NaN
20,10437,0,2.0,0,7.0,2.0,"[42, 46]",NaN
38,10715,0,2.0,0,6.0,2.0,"[31, 33]",NaN
68,10315,1,2.0,0,8.0,2.0,"[1, 30]",NaN
80,10439,1,2.0,0,9.0,2.0,"[39, 41]",NaN
...,...,...,...,...,...,...,...,...
3195,10315,6,2.0,6,7.0,NaN,NaN,10.5
3225,10717,6,2.0,6,6.0,NaN,NaN,9.0
3243,10997,6,2.0,6,7.0,NaN,NaN,10.5
3252,10247,7,2.0,6,8.0,NaN,NaN,12.0


In [161]:
whole_solution_df.query('n_couriers == 2 & employee_count == 2')

,area,period,n_couriers,day,deliveries,employee_count,employees_assign,cost_outsource
10,10317,0,2.0,0,6.0,2.0,"[1, 26]",NaN
20,10437,0,2.0,0,7.0,2.0,"[42, 46]",NaN
38,10715,0,2.0,0,6.0,2.0,"[31, 33]",NaN
68,10315,1,2.0,0,8.0,2.0,"[1, 30]",NaN
80,10439,1,2.0,0,9.0,2.0,"[39, 41]",NaN
...,...,...,...,...,...,...,...,...
2979,10589,2,2.0,6,6.0,2.0,"[34, 43]",NaN
3073,10243,4,2.0,6,6.0,2.0,"[27, 41]",NaN
3085,10407,4,2.0,6,6.0,2.0,"[47, 51]",NaN
3135,10249,5,2.0,6,6.0,2.0,"[4, 32]",NaN
